In [10]:
import numpy as np
from simglucose.simulation.env import T1DSimEnv
from simglucose.controller.basal_bolus_ctrller import BBController
from simglucose.sensor.cgm import CGMSensor
from simglucose.actuator.pump import InsulinPump
from simglucose.patient.t1dpatient import T1DPatient
from simglucose.simulation.scenario_gen import RandomScenario
from simglucose.simulation.sim_engine import SimObj, sim
from datetime import timedelta, datetime
import pandas as pd

# Initialize random seed for reproducibility
np.random.seed(42)

# Parameters
path = './results'
start_time = datetime.combine(datetime.now().date(), datetime.min.time())

categories = ['child', 'adolescent', 'adult']
patients_per_category = 10

all_dfs = []

for category in categories:
    for i in range(1, patients_per_category + 1):
        patient_name = f"{category}#{i:03d}"
        
        # Generate synthetic metadata
        if category == 'child':
            age = np.random.randint(5, 13)
            bmi = np.round(np.clip(np.random.normal(17, 2), 12, 25), 1)
            heart_rate = np.random.randint(70, 101)
        elif category == 'adolescent':
            age = np.random.randint(13, 20)
            bmi = np.round(np.clip(np.random.normal(21, 3), 15, 30), 1)
            heart_rate = np.random.randint(60, 101)
        else:
            age = np.random.randint(20, 61)
            bmi = np.round(np.clip(np.random.normal(25, 5), 18, 40), 1)
            heart_rate = np.random.randint(60, 91)
        gender = np.random.choice(['Male', 'Female'])
        
        # Create patient
        patient = T1DPatient.withName(patient_name)
        
        # Create components with unique seeds
        seed = i + (hash(category) % 1000)
        sensor = CGMSensor.withName('Dexcom', seed=seed)
        pump = InsulinPump.withName('Insulet')
        scenario = RandomScenario(start_time=start_time, seed=seed)
        env = T1DSimEnv(patient, sensor, pump, scenario)
        controller = BBController()
        
        # Run simulation
        sim_obj = SimObj(env, controller, timedelta(days=1), animate=False, path=path)
        results = sim(sim_obj)
        
        # Process results: Convert index to column and create a complete minute-level timeline
        results.reset_index(inplace=True)  # 'Time' becomes a column
        
        # Create a complete minute-level DatetimeIndex for one day
        complete_index = pd.date_range(start=start_time, periods=1440, freq='min')
        
        # Set 'Time' as index for merging
        results.set_index('Time', inplace=True)
        
        # Reindex to have one entry per minute. This will introduce NaNs where no observation existed.
        results = results.reindex(complete_index)
        
        # Ensure the index name is 'Time'
        results.index.name = 'Time'
        
        # Fill any NaN values using forward fill and backward fill
        results.ffill(inplace=True)
        results.bfill(inplace=True)
        
        # Create the 'Minute' column as the number of minutes elapsed from start_time
        results.reset_index(inplace=True)
        results['Minute'] = (results['Time'] - pd.Timestamp(start_time)).dt.total_seconds() // 60
        
        # Keep only desired columns and rename them
        results = results[['Minute', 'BG', 'insulin', 'CHO']]
        results.rename(columns={
            'BG': 'Glucose',
            'insulin': 'Insulin',
            'CHO': 'CarbsIntake'
        }, inplace=True)
        
        # Add metadata
        results['PatientID'] = patient_name
        results['Age'] = age
        results['Gender'] = gender
        results['BMI'] = bmi
        results['HeartRate'] = heart_rate
        
        # Reorder columns to have PatientID first
        column_order = ['PatientID', 'Minute', 'Glucose', 'Insulin', 'CarbsIntake', 'Age', 'Gender', 'BMI', 'HeartRate']
        results = results[column_order]
        
        all_dfs.append(results)

# Combine all data
dataset = pd.concat(all_dfs, ignore_index=True)

# Save dataset
dataset.to_csv('glucose_dataset.csv', index=False)

print(f"Dataset created with {len(dataset)} entries.")

Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process ID: 20256
Simulation starts ...
Simulation Completed!
Process 

In [11]:
dataset

,PatientID,Minute,Glucose,Insulin,CarbsIntake,Age,Gender,BMI,HeartRate
0,child#001,0.0,141.204660,0.006575,0.0,11,Male,15.9,77
1,child#001,1.0,141.204660,0.006575,0.0,11,Male,15.9,77
2,child#001,2.0,141.204660,0.006575,0.0,11,Male,15.9,77
3,child#001,3.0,141.204660,0.006575,0.0,11,Male,15.9,77
4,child#001,4.0,141.204660,0.006575,0.0,11,Male,15.9,77
...,...,...,...,...,...,...,...,...,...
43195,adult#010,1435.0,132.171220,0.016975,0.0,33,Male,18.0,62
43196,adult#010,1436.0,132.171220,0.016975,0.0,33,Male,18.0,62
43197,adult#010,1437.0,131.150467,0.016975,0.0,33,Male,18.0,62
43198,adult#010,1438.0,131.150467,0.016975,0.0,33,Male,18.0,62


In [14]:
df = pd.DataFrame(dataset)

In [16]:
df.to_csv('Dataset_glucosio_finale.csv', index=False)  # Saves without the index column


In [18]:
df_loaded = pd.read_csv('Dataset_glucosio_finale.csv')
print(df_loaded)

       PatientID  Minute     Glucose   Insulin  CarbsIntake  Age Gender   BMI  \
0      child#001     0.0  141.204660  0.006575          0.0   11   Male  15.9   
1      child#001     1.0  141.204660  0.006575          0.0   11   Male  15.9   
2      child#001     2.0  141.204660  0.006575          0.0   11   Male  15.9   
3      child#001     3.0  141.204660  0.006575          0.0   11   Male  15.9   
4      child#001     4.0  141.204660  0.006575          0.0   11   Male  15.9   
...          ...     ...         ...       ...          ...  ...    ...   ...   
43195  adult#010  1435.0  132.171220  0.016975          0.0   33   Male  18.0   
43196  adult#010  1436.0  132.171220  0.016975          0.0   33   Male  18.0   
43197  adult#010  1437.0  131.150467  0.016975          0.0   33   Male  18.0   
43198  adult#010  1438.0  131.150467  0.016975          0.0   33   Male  18.0   
43199  adult#010  1439.0  131.150467  0.016975          0.0   33   Male  18.0   

       HeartRate  
0       